In [1]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score
import numpy as np
import pandas as pd


data = pd.read_csv('C:\\Users\\ydhar\\Downloads\\USA_Housing.csv') 

In [2]:
X = data.drop('Price', axis=1)
y = data['Price']
X

,Avg. Area Income,Avg. Area House Age,Avg. Area Number of Rooms,Avg. Area Number of Bedrooms,Area Population
0,79545.45857,5.682861,7.009188,4.09,23086.80050
1,79248.64245,6.002900,6.730821,3.09,40173.07217
2,61287.06718,5.865890,8.512727,5.13,36882.15940
3,63345.24005,7.188236,5.586729,3.26,34310.24283
4,59982.19723,5.040555,7.839388,4.23,26354.10947
...,...,...,...,...,...
4995,60567.94414,7.830362,6.137356,3.46,22837.36103
4996,78491.27543,6.999135,6.576763,4.02,25616.11549
4997,63390.68689,7.250591,4.805081,2.13,33266.14549
4998,68001.33124,5.534388,7.130144,5.44,42625.62016


In [3]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
X=scaler.fit_transform(X)
X

array([[ 1.02865969, -0.29692705,  0.02127433,  0.08806222, -1.31759867],
       [ 1.00080775,  0.02590164, -0.25550611, -0.72230146,  0.40399945],
       [-0.68462915, -0.11230283,  1.5162435 ,  0.93084045,  0.07240989],
       ...,
       [-0.48723454,  1.28447022, -2.17026949, -1.50025059, -0.29193658],
       [-0.05459152, -0.44669439,  0.14154061,  1.18205319,  0.65111608],
       [-0.28831272,  0.01521477, -0.19434166,  0.07185495,  1.04162464]])

In [4]:
kf = KFold(n_splits=5)
r2_scores = []
beta_matrices = []

for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

In [5]:
 model = LinearRegression()
 model.fit(X_train, y_train)

LinearRegression()

In [6]:
 beta = model.coef_
 y_pred = model.predict(X_test)
 y_pred

array([1457655.14509262, 1298104.16714785, 1650238.22147915,
       1605071.83890936, 1404788.08286058, 1543416.35563577,
       1364105.23837493, 1464953.62802342, 1094094.64619444,
       1158087.94530995, 1511152.56199185, 1046492.34578515,
       1146556.39584728, 1259132.86913504,  951852.22073643,
       1336707.0374408 , 1625733.18779646, 1292949.70767621,
       1331550.41492348, 1741540.8736312 , 1169156.93563025,
       1635074.26335467, 1413875.04178438, 1262012.73379097,
       1385644.43638768,  859121.05235639,  798820.59132736,
        915218.74460299,  942636.20977044, 1506585.09649504,
       1848833.64327817, 1244268.717695  , 1592507.19031079,
        887720.74173755,  504984.73986602,  657861.75613857,
       1422024.01677379, 1533974.6363799 , 1188949.94231217,
       1657671.33609735,  839103.44475237, 1078152.94258651,
       1161935.02701188, 1145551.70447918, 1160261.74446638,
       1251304.61747991, 1194844.85231176,  967996.37835262,
       1126122.20316288,

In [9]:
r2 = r2_score(y_test, y_pred)
r2_scores.append(r2)
beta_matrices.append(beta)


In [10]:
best_beta = beta_matrices[np.argmax(r2_scores)]


train_size = int(0.7 * len(X))
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
r2 = r2_score(y_test, y_pred)
r2

0.9176499755975894

In [14]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDRegressor
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split dataset into training (56%), validation (14%), and test (30%) sets
X_train, X_temp, y_train, y_temp = train_test_split(X_scaled, y, test_size=0.44, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.68, random_state=42)

# Define learning rates to test
learning_rates = [0.001, 0.01, 0.1, 1]
best_r2_val = -np.inf
best_model = None

for lr in learning_rates:
    # Using SGDRegressor for gradient descent optimization
    model = SGDRegressor(learning_rate='constant', eta0=lr, max_iter=1000)
    model.fit(X_train, y_train)
    
    # Validation performance
    y_val_pred = model.predict(X_val)
    r2_val = r2_score(y_val, y_val_pred)
    
    # Test performance
    y_test_pred = model.predict(X_test)
    r2_test = r2_score(y_test, y_test_pred)
    
    print(f"Learning rate: {lr}, R2 score on validation set: {r2_val}, R2 score on test set: {r2_test}")
    
    # Track the best model
    if r2_val > best_r2_val:
        best_r2_val = r2_val
        best_model = model

# Use the best model
print("Best model coefficients:", best_model.coef_)


Learning rate: 0.001, R2 score on validation set: 0.9202473736651289, R2 score on test set: 0.9132304311481162
Learning rate: 0.01, R2 score on validation set: 0.9194279572340154, R2 score on test set: 0.911622072386859
Learning rate: 0.1, R2 score on validation set: 0.8669939900832053, R2 score on test set: 0.8571664879312231
Learning rate: 1, R2 score on validation set: -82354260314935.05, R2 score on test set: -90364296207439.53
Best model coefficients: [231874.06994562 165202.17990892 121665.30762209    947.73374676
 151761.34202432]
